In [1]:
import cv2
import dlib
from imutils import face_utils
from scipy.spatial import distance
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os
from tqdm import tqdm  # Import tqdm for progress bar

# Function to calculate EAR
def eye_aspect_ratio(eye):
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])
    C = distance.euclidean(eye[0], eye[3])
    return (A + B) / (2.0 * C)

# Constants
THRESH = 0.25  # EAR threshold
FRAME_CHECK = 20  # Frames required for drowsiness
flag = 0  # Drowsiness flag

# Initialize models
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("models\shape_predictor_68_face_landmarks.dat")
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

# Folder paths
drowsy_folder = r'C:\Users\MEENAKSHI\Downloads\created-ds\created-ds-closed'  # Folder containing drowsy videos (label 1)
not_drowsy_folder = r'C:\Users\MEENAKSHI\Downloads\created-ds\created-ds-open'  # Folder containing not drowsy videos (label 0)

# Get all video file paths from the folders
drowsy_videos = [(os.path.join(drowsy_folder, f), 1) for f in os.listdir(drowsy_folder) if f.endswith(('.mp4', '.avi'))]
not_drowsy_videos = [(os.path.join(not_drowsy_folder, f), 0) for f in os.listdir(not_drowsy_folder) if f.endswith(('.mp4', '.avi'))]

# Dataset: List of tuples (video_path, ground_truth)
datadataset = drowsy_videos + not_drowsy_videos

# Ensure results storage
ground_truths = []
predictions = []

# Initialize tqdm progress bar for dataset processing
with tqdm(total=len(datadataset), desc="Processing Videos", unit="video") as pbar:

    # Evaluate each video
    for video_path, ground_truth in datadataset:
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print(f"Error: Unable to open video {video_path}")
            continue  # Skip this video if not readable

        ground_truths.append(ground_truth)  # Append the ground truth for this video

        drowsy_detected = 0
        flag = 0  # Reset flag for each video

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = detector(gray)

            for face in faces:
                shape = predictor(gray, face)
                shape = face_utils.shape_to_np(shape)
                leftEye = shape[lStart:lEnd]
                rightEye = shape[rStart:rEnd]
                leftEAR = eye_aspect_ratio(leftEye)
                rightEAR = eye_aspect_ratio(rightEye)
                ear = (leftEAR + rightEAR) / 2.0

                if ear < THRESH:
                    flag += 1
                else:
                    flag = 0

                if flag >= FRAME_CHECK:
                    drowsy_detected = 1
                    break

            if drowsy_detected:
                break

        predictions.append(drowsy_detected)  # Append the prediction for this video
        cap.release()

        # Update progress bar
        pbar.update(1)

# Verify lengths of ground_truths and predictions
assert len(ground_truths) == len(predictions), "Mismatch in ground truth and predictions length!"

# Calculate accuDrowsiness_Detection-master/created-data-accuracy.ipynbracy metrics
accuracy = accuracy_score(ground_truths, predictions)
precision = precision_score(ground_truths, predictions, zero_division=0)
recall = recall_score(ground_truths, predictions, zero_division=0)
f1 = f1_score(ground_truths, predictions, zero_division=0)

# Display results
print("Evaluation Metrics:")
print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision * 100:.2f}%")
print(f"Recall: {recall * 100:.2f}%")
print(f"F1-Score: {f1 * 100:.2f}%")


Processing Videos: 100%|██████████| 151/151 [23:22<00:00,  9.29s/video]

Evaluation Metrics:
Accuracy: 94.04%
Precision: 98.57%
Recall: 89.61%
F1-Score: 93.88%
